In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
links=pd.read_csv('links.csv')

In [3]:
links['Links']

0      https://www.imf.org/en/News/Articles/2024/10/2...
1      https://www.imf.org/en/News/Articles/2024/04/1...
2      https://www.imf.org/en/News/Articles/2023/10/1...
3      https://www.imf.org/en/News/Articles/2023/04/1...
4      https://www.imf.org/en/News/Articles/2022/10/1...
                             ...                        
168    https://www.imf.org/en/News/Articles/2015/09/2...
169    https://www.imf.org/en/News/Articles/2015/09/2...
170    https://www.imf.org/en/News/Articles/2015/09/2...
171    https://www.imf.org/en/News/Articles/2015/09/2...
172    https://www.imf.org/en/News/Articles/2015/09/2...
Name: Links, Length: 173, dtype: object

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Data storage
data = []

# Iterate over links
for link in links['Links']:
    try:
        # Fetch the page content
        response = requests.get(link)
        response.raise_for_status()  # Ensure the request was successful
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the target <section> tag
        section = soup.find('section')
        
        if section:
            # Extract the title from <h2>
            title = section.find('h2').get_text(strip=True) if section.find('h2') else "No Title"
            
            # Extract the date from <p class="date">
            date_element = section.find('p', class_='date')
            date = date_element.get_text(strip=True) if date_element else "No Date"
            
            # Extract all <p> tags and combine their text into one string (excluding the date)
            subtext = " ".join([p.get_text(strip=True) for p in section.find_all('p') if p != date_element])
            
            # Add to data list
            data.append({'Title': title, 'Date': date, 'Subtext': subtext})
        else:
            # Handle missing <section>
            data.append({'Title': "No Section Found", 'Date': "No Date Found", 'Subtext': "No Subtext Found"})
    
    except Exception as e:
        # Handle errors and continue with the next link
        print(f"Error processing {link}: {e}")
        data.append({'Title': "Error", 'Date': "Error", 'Subtext': str(e)})

# Convert data into a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

                                                 Title                Date  \
0    Intergovernmental Group of Twenty-Four on Inte...    October 22, 2024   
1    Intergovernmental Group of Twenty-Four on Inte...      April 16, 2024   
2    Intergovernmental Group of Twenty-Four on Inte...    October 10, 2023   
3    Intergovernmental Group of Twenty-Four on Inte...      April 11, 2023   
4    Intergovernmental Group of Twenty-Four on Inte...    October 11, 2022   
..                                                 ...                 ...   
168  Communiqué of the Interim Committee of the Boa...  September 21, 1997   
169                    Group of Twenty Four Communiqué  September 20, 1997   
170  Communiqué of the Ministers and Governors of t...      April 28, 1997   
171                       Interim Committee Communiqué      April 28, 1997   
172  Intergovernmental Group of Twenty-Four on Inte...      April 27, 1997   

                                               Subtext  
0    1

In [5]:
df.to_csv("scraped_data.csv", index=False)